## Chapter 4 -  Training Models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split

In [2]:
# Ingest, preprocessing
df = pd.read_csv('Advertising.csv', index_col=0)

X1 = df.iloc[:,:1]
df1 = df.iloc[:,[0,1,2,3]]

X2 = df.iloc[:,:3]
df2 = df.iloc[:, [0,3]]
y = df.iloc[:, 3]

In [3]:
# For testing
# display(df.head())
# display(X1.head())
# display(df1.head())
# display(X2.head())
# display(y.head())

### Validity of the Coefficient Estimates
The linear regression model assumes that the true relationship between $x$ and $y$ is $y=f(x)+\epsilon$ where $\epsilon$ is a mean-zero random error term. For univariate linear regression,

$$y = f(\mathbf x) + \epsilon =\beta_0 + \beta_1x_1 + \epsilon$$ 

and for multivariate linear regression with $p$ variables, 

$$y=f(\mathbf x) + \epsilon = \beta_0 + \beta_1x_{1} + \beta_2x_{2}+ \cdots + \beta_px_{p} + \epsilon$$ 

and $p=1$ for the univariate case.

$\beta_0$ is the intercept term - the value of $y$ when $x_j=0 \,\,\forall j \in \{1,\cdots,p\}$. $\beta_j$ is the average increase in $y$ associated with one unit increase in $x_j$. The error term, $\epsilon$ is a catch-all for what is missed with the model: there may be other variables that cause a variation in $y$, and there may be measurement error. This error term is independent of $\mathbf x$.

### Univariate Linear Regression

In [7]:
# Univariate case
X1 = sm.add_constant(X1)
uni_reg = sm.OLS(y, X1)
univ_results = uni_reg.fit()
# display(univ_results.summary2())
print(univ_results.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.0326      0.458     15.360      0.000       6.130       7.935
TV             0.0475      0.003     17.668      0.000       0.042       0.053


In the univariate case, the model $y = \beta_0 + \beta_1x_1 + \epsilon$ is the <u>population regression line</u>, and the linear approximation model $\hat{y} = \hat{\beta_0} + \hat{\beta_1}x_1$ is the <u>least squares line</u>. In reality, the true relationship between $\mathbf x$ and $y$ is generally not known, but the parameters of the least squares line can be computed using the closed-form solution using the observed data. Fundamentally, this is the statistical approach where we are <b>using observations from an experiment/sample to estimate characteristics of a large population</b>.

In the estimation of the mean problem, we want to find the mean $\mu_K$ of an unknown variable, say $K$. $\mu_K$ is unknown but we have $n$ observations from the population and we estimate $\mu$ using the sample mean $\bar k = \sum_{i=1}^n k_i$. The sample mean $\bar k$ and the population mean $\mu_K$ are different, but in general the sample mean will provide a good estimate of the population mean.

Using $\hat{\mu_K} = \bar k$ to estimate $\mu_K$ is similar to using $\hat{\beta_0}$ and $\hat{\beta_1}$ to estimate the model parameters $\beta_0$ and $\beta_1$. They define the estimated least squares line used to estimate the population regression line.

The sample mean $\hat{\mu_K}=\bar k$ is an <u>unbiased</u> estimator of the population mean $\mu_K$. This means that on average (across many estimates of many samples), we expect $\hat{\mu_K}=\mu_K$. Specifically, when we measure $\hat{\mu_k}$ many times and average the estimates, we will sometimes overestimate $\mu_K$ and sometimes underestimate $\mu_K$. However, by averaging out a large number of estimates, the average will exactly equals $\mu_K$. 

Hence, an unbiased estimator does not systematically overestimate or underestimate the true parameter. The property of unbiasedness also holds true for the least squares estimates in the regression model.

In the estimation of $\mu_K$ problem, if some estimates $\hat\mu_K$ are above and some are below the true population mean, how, then can we establish how far is a single estimate $\hat\mu_K$ from the true parameter? We use the <u>standard error</u> of $\hat\mu_K$, $\text{SE}(\hat\mu_K)$ to help us:

$$\text{Var}(\hat\mu_K) = \text{SE}(\hat\mu_K)^2 = \frac{\sigma_K^2}{n}$$

where $\sigma_K$ is the standard deviation of $K$ or the population standard deviation, and $n$ is the sample size. Observe from the formula that the standard error decreases as $n$ increases. The more observations we have in a sample, the smaller the standard error of $\hat\mu_K$. 

In least squares estimation, we want to compute the standard errors associated with $\hat{\beta_0}$ and $\hat{\beta_1}$ and they are:

$$\text{SE}(\beta_0)^2 = \sigma^2\begin{bmatrix}\frac 1n + \frac{\bar x^2}{\sum_{i=1}^n(x_i-\bar x)^2}\end{bmatrix}$$

$$\text{SE}(\beta_1)^2 = \frac{\sigma^2}{\sum_{i=1}^n(x_i-\bar x)^2}$$

where $\sigma^2 = \text{Var}(\epsilon)$. The assumption is that the errors $\epsilon_i$ for each observation is uncorrelated and have the same variance $\sigma^2$. $\sigma^2$ can be estimated from the data and it is known as the <u>residual standard error</u>, RSE and is given by the formula

$$\text{RSE} = \sqrt{\frac{\text{RSS}}{n-2}}$$

Assuming that the errors are normally distributed, standard errors can be used to compute <u>confidence intervals</u> (C.I.). a 95% confidence interval is defined as a range of values such that with 95% probability, the range will contain the true unknown value of the parameter. The range is defined as the lower and upper limits computed from the sample of data. In least squares estimation, the 95% C.I. for $\beta_1$ is:

$$\beta_1 \pm 1.96\times \text{SE}(\hat{\beta_1})$$

and thus there is a 95% chance that the true value of $\beta_1$ lies in the interval:

$$\begin{bmatrix}\beta_1 - 1.96\times \text{SE}(\hat{\beta_1}), \beta_1 + 1.96\times \text{SE}(\hat{\beta_1})\end{bmatrix}$$

Similarly, the 95% C.I. for $\beta_0$ is:
$$\begin{bmatrix}\beta_0 - 1.96\times \text{SE}(\hat{\beta_0}), \beta_1 + 1.96\times \text{SE}(\hat{\beta_0})\end{bmatrix}$$

(The value $1.96$ is the corresponding value where $p=0.975$ in the normal table, assuming $n$ is large. Otherwise, use the 97.5% quantile of a $t$-distribution with $n-2$ degrees of freedom.)

In [8]:
print(univ_results.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.0326      0.458     15.360      0.000       6.130       7.935
TV             0.0475      0.003     17.668      0.000       0.042       0.053


In the Advertising example, the 95% C.I. for $\beta_0$ is $[6.130, 7.935]$ and the 95% C.I. for $\beta_1$ is $[0.042, 0.053]$. Hence, with no advertising, sales will fall on average between 6130 and 7940 units. Each \$1000 increase in advertising will result in an average increase in sales of between 42 and 53 units.

Standard errors can also be used to perform <u>hypothesis testing</u> on the coefficients. The null and alternative hypothesis are:
$$H_0:\text{There is no relationship between }x\text{ and }y$$
$$H_1:\text{There is some relationship between }x\text{ and }y$$

Mathematically, 
$$H_0:\beta_1=0$$
$$H_1:\beta_1 \neq 0$$

If the null is true, then the model simply reduces to $y=\beta_0 + \epsilon$, with the conclusion that there is no relationship. 

How large must $\beta_1$ be to reject the null? It depends on $\text{SE}(\hat{\beta_1})$, relative to (\hat{\beta_1}). To find the extent of how far $\beta_1$ is from $0$, we compute the $t$-statistic:

$$t=\frac{\hat{\beta_1}-0}{\text{SE}(\beta_1)}$$

which measures how many standard deviations is $\hat \beta_1$ from $0$. Consequently, the $p$-value is the probability of observing a value larger than or equal to $|t|$, given the null hypothesis is true. 

We can also use the $p$-value to decide whether to reject the null. A small $p$-value indicates that it is <b>unlikely to observe no relationship</b> between $x$ and $y$, and we can infer that there is indeed a relationship between the variables and the predictor. We reject the null hypothesis and conclude that there is indeed a relationship between the variables and the response. Typical thresholds of the $p$-value is $0.05$ or $0.01$.

In [9]:
print(univ_results.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.0326      0.458     15.360      0.000       6.130       7.935
TV             0.0475      0.003     17.668      0.000       0.042       0.053


In the Advertising example, since the $p$-value for both $\hat \beta_0$ and $\hat \beta_1$ are both very small (virtually $0$), we can reject the null in favour of the alternative: both $\beta_0 \neq 0$ and $\beta_1 \neq 0$ must be true.

### Accuracy of the Model

Now that we have rejected the null hypothesis in favour of the alternative, the next step is to quantify the extent which the model fits the data. This is measured using the residual standard error (RSE) and <u>the $R^2$ statistic</u>. 

<b>Residual Standard Error (RSE)</b> - Recall that the model contains an error term $\epsilon$. Because of these error terms, even if we know the true regression line, we cannot perfectly predict $y$ from $x$. The RSE is an estimate of the standard deviation of $\epsilon$. Roughly, it is the average amount that the response will deviate from the true regression line, and is computed as:

$$\text{RSE} = \sqrt{\frac{\text{RSS}}{n-2}} = \sqrt{\frac{1}{n-2}\sum_{i=1}^n \begin{bmatrix} \hat{y^{(i)}} - y^{(i)})\end{bmatrix}^2}$$

In [24]:
print('RSE=' + str(np.sqrt(univ_results.ssr/(univ_results.nobs-2))))
print('mean=' + str(y.mean()))

RSE=3.258656368650463
mean=14.0225


In the Advertising example, the RSE is $3.26$. This means actual sales deviate from the true regression line by approximately 3260 units. Or, even if the model were correct, the prediction wll still be off by 3260 on average. How significant this is depends on the ovarall value. If the mean is 14000 units then 3260 will mean an estimation error of about (3260/14000)=23%.

The RSE is a measure of lack of fit of the model to de data. Since it depends on RSS, notice that a small RSE indicates that the model is a good fit of the data, while a large RSE indicates the model does not fit the data well.

<b>$R^2$ statistic</b> - the RSE provides an absolute measure of lack of fit of the model to the data. But it is measured in units of $y$. To overcome this, we use the $R^2$ statistic, which takes the form of a proportion - the proportion of variance explained, and is independent of $y$. It is calculated as:

$$R^2 = \frac{\text{TSS} - \text{RSS}}{\text{TSS}} = 1 - \frac{\text{RSS}}{\text{TSS}}$$

where $\text{TSS}$ is defined as total sum of squares, $\text{TSS} = (y^{(i)}-\bar y)^2$ and $\text{RSS} = \begin{pmatrix} \hat{y^{(i)}} - y^{(i)}\end{pmatrix}^2$. TSS is the total variance in the response $y$ and can be thought of the amount of variability before applying the regression model. Hence, $\text{TSS} - \text{RSS}$ is the variance explained amount of variability in the response that is explained (or removed) by performing the regression, and $R^2$ measures the proportion of variability that can be explained using $x$. An $R^2$ close to $1$ indicates a large proportion of variability is explained by the regression while a value close to $0$ indicates that the regression did not explain much of the variability in the response.

In [32]:
print(univ_results.summary().tables[0])
print('rsquared=' + str(univ_results.rsquared))

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     312.1
Date:                Tue, 12 May 2020   Prob (F-statistic):           1.47e-42
Time:                        18:08:41   Log-Likelihood:                -519.05
No. Observations:                 200   AIC:                             1042.
Df Residuals:                     198   BIC:                             1049.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
rsquared=0.611875050850071


In the advertising example, since $R^2=0.612$, about 61% of variability in sales is explained by a linear regression on the TV variable.

The $R^2$ is a measure of the linear relationship between $x$ and $y$. Recall that correlation is also a measure of relationship between $x$ and $y$:

$$\text{Cor}(x,y) = r^2=\frac{\sum_{i=1}^n (x^{(i)}-\bar x)
(y^{(i)}-\bar y)}{\sqrt{\sum_{i=1}^n (x^{(i)}-\bar x)^2}\sqrt{\sum_{i=1}^n(y^{(i)}-\bar y)^2}}$$

This suggest that we can use correlation to also measure the fit of the linear model. In fact, in univariate linear regression, $r^2 = R^2$. However, this does not apply to the multivariate case.

In [48]:
print(df.corr())
print()
print(univ_results.summary().tables[1])
print()
# Consider univariate linear regression on the other 2 variables
X11 = df.iloc[:,[2]]
univ_results2 = sm.OLS(y, sm.add_constant(X11)).fit()
print(univ_results2.summary().tables[1])
print()
X12 = df.iloc[:,[1]]
uni_reg3 = sm.OLS(y, sm.add_constant(X12)).fit()
print(univ_results3.summary().tables[1])


                 TV     radio  newspaper     sales
TV         1.000000  0.054809   0.056648  0.782224
radio      0.054809  1.000000   0.354104  0.576223
newspaper  0.056648  0.354104   1.000000  0.228299
sales      0.782224  0.576223   0.228299  1.000000

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.0326      0.458     15.360      0.000       6.130       7.935
TV             0.0475      0.003     17.668      0.000       0.042       0.053

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.3514      0.621     19.876      0.000      11.126      13.577
newspaper      0.0547      0.017      3.300      0.001       0.022       0.087

                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

### Multivariate Linear Regression

In [35]:
# Multivariate case
X2 = sm.add_constant(X2)
multiv_reg = sm.OLS(y, X2)
multiv_results = multiv_reg.fit()
print(multiv_results.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
radio          0.1885      0.009     21.893      0.000       0.172       0.206
newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011


In the advertising example, the above table shows the coefficient estimates when TV, radio and newspaper advertising budgets are used to predict product sales. For a $1000 increase in radio advertising, holding the rest of the variables constant, we expect to see a 189 unit increase in sales.

Notice that the coefficient for `newspaper` is near-zero in the multivariate regression result (and has a large $p$-value), while it is relatively large when it alone is regressed on `sales`. Does these conflicting results make sense?



<b>Hypothesis Testing II - Multivariate Regression</b>

In the multivariate case, we set the null to be that all the coefficients of the regression model are $0$.

Mathematically, 
$$H_0:\beta_1 = \beta_2 = \cdots = \beta_p =0$$
$$H_1:\text{Any }\beta_j \,\, , j \in \{1,\cdots,p\} \text{ is non-zero}$$

Now, this is tested using the $F$-statistic:

$$F = \frac{\frac{\text{TSS} - \text{RSS}}{p}}{\frac{\text{RSS}}{n-p-1}}$$

The definition of TSS and RSS are the same as of univariate regression. If there is no relationship between the response and the variables, then the $F$-statistic is close to $1$. Otherise $F>1$. So $F>1$ is the rule to use to reject the null in favour of the alternative (there is indeed relationship between $x$ and $y$.

A large $F$-statistic suggests that at least one of the variables is related to the target variable.

Similarly, we can use the $p$-value to determine whether to reject the null.